In [1]:
import joblib
import sys
sys.path.extend(['../../../config', '../../app/utils'])

import pandas as pd
import numpy as np

from api import Features
from model import AdvancedModel
from preprocessing_utils import DtypeTransformer

RANDOM_STATE = 321

## Сравнение результатов предсказаний ядром и моделью.
- Загрузим данные
- Сделаем предсказания ядром и моделью
- Сравним результаты

In [2]:
# Загрузка данных
test_set = pd.read_csv('test_set.csv', index_col='sk_id_curr', low_memory=False)
test_set_ = test_set.iloc[:1000].copy()

In [3]:
# Загрузка и инициализация моделей
lgbm_model = joblib.load('lgbm_pipeline.pkl')
core_model = AdvancedModel(model_path='lgbm_pipeline.pkl')

In [ ]:
# Предсказания
test_set_['proba_lgbm'] = lgbm_model.predict_proba(test_set_)[:,1]
test_set_['core_proba'] = test_set_.drop('proba_lgbm', axis=1)\
    .apply(lambda x: core_model.get_scoring_result(x).proba, axis=1
)

In [5]:
# Сравнение предсказаний
all(test_set_['proba_lgbm'].round(4) == test_set_['core_proba'].round(4))

True

In [6]:
# Для наглядности
test_set_.head(3)

,ext_source_1,ext_source_2,ext_source_3,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,days_birth,days_employed,...,flag_document_14,flag_document_15,flag_document_16,flag_document_17,flag_document_18,flag_document_19,flag_document_20,flag_document_21,proba_lgbm,core_proba
sk_id_curr,,,,,,,,,,,,,,,,,,,,,
202897,1.0,1.0,1.0,0,135000.0,553500.0,28390.5,553500.0,17472,3719,...,False,False,False,False,False,False,False,False,0.168171,0.168171
202899,NaN,1.0,1.0,0,180000.0,247500.0,18139.5,247500.0,20578,1780,...,False,False,False,False,False,False,False,False,0.414151,0.414151
202907,NaN,1.0,NaN,0,135000.0,237024.0,12231.0,180000.0,18301,246,...,False,False,False,False,False,False,False,False,0.331677,0.331677


Предсказания моделью и ядром равны.

## Финансовый результат предсказаний и одобрения сумм.

In [7]:
# Загрузим данные
data = pd.read_csv('train_set.csv', index_col='sk_id_curr')

- **Для воспроизводимости**: то - как изначально был разделён тренировочный датасет.   
Чтобы уложиться в лимиты GitLab по размеру загружаемых файлов (100 MiB) - пришлось перезаписать.

In [8]:
# Получим результаты скоринга и извлечём необходимые данные
data['result'] = data.drop('target', axis=1).apply(lambda x: core_model.get_scoring_result(x), axis=1)
data['decision'] = data['result'].apply(lambda x: x.decision.name)
data['amount'] = data['result'].apply(lambda x: x.amount)
data['proba'] = data['result'].apply(lambda x: x.proba)

Насчитаем статистики по результатам скоринга.

In [9]:
annuity_rate = 0.15

result = data.pivot_table(
    index=['decision', 'amount'], 
    values=['proba', 'amt_credit'], 
    aggfunc={'proba': 'mean', 'amt_credit': 'count'}
).reset_index().rename(columns={'amt_credit': 'count'})

result['defaulters'] = result['count'] * result['proba']
result['paid'] = result['count'] - result['defaulters']
result['loss'] = result['defaulters'] * result['amount'] 
result['gross'] = result['paid'] * result['amount'] * annuity_rate
result['profit'] = result['gross'] - result['loss']

display(result)
print(f'Финансовый результат: {result["profit"].sum()}')

,decision,amount,count,proba,defaulters,paid,loss,gross,profit
0,accepted,20000,11,0.205824,2.264064,8.735936,4.528129e+04,2.620781e+04,-1.907348e+04
1,accepted,50000,2152,0.169282,364.295580,1787.704420,1.821478e+07,1.340778e+07,-4.806996e+06
2,accepted,100000,498,0.164540,81.940688,416.059312,8.194069e+06,6.240890e+06,-1.953179e+06
3,accepted,250000,494,0.071006,35.076821,458.923179,8.769205e+06,1.720962e+07,8.440414e+06
4,declined,0,6071,0.493242,2994.473917,3076.526083,0.000000e+00,0.000000e+00,0.000000e+00


Финансовый результат: 1661165.352817228


Нам удалось выделить с помощью калькулятора:

- Заёмщиков с очень низким дефолтом 
- Отказать в выдаче кредитов потенциальным заёмщикам с очень высоким дефолтом.
